In [1]:
# Data from here:
# https://www.ecdc.europa.eu/en/publications-data/download-todays-data-geographic-distribution-covid-19-cases-worldwide
import pandas as pd
import glob
import os

In [2]:
# READ, RENAME
covid = pd.read_excel("COVID-19-geographic-disbtribution-worldwide-2020-08-27.xlsx") 
# REPLACED WITH THE CODE BELOW THAT EXTRACTS THE LATEST .xlsx FILE IN THE FOLDER
covid
# list_of_files = glob.glob('C:\\Users\\nikan\\Desktop\\COVID_daily/*xlsx') # * means all if need specific format then *.csv
# latest_file = max(list_of_files, key=os.path.getctime)
# print(latest_file)
# covid = pd.read_excel(latest_file) 

,dateRep,day,month,year,cases,deaths,countriesAndTerritories,geoId,countryterritoryCode,popData2019,continentExp,Cumulative_number_for_14_days_of_COVID-19_cases_per_100000
0,2020-08-27,27,8,2020,55,4,Afghanistan,AF,AFG,38041757.0,Asia,2.053007
1,2020-08-26,26,8,2020,1,0,Afghanistan,AF,AFG,38041757.0,Asia,2.108210
2,2020-08-25,25,8,2020,71,10,Afghanistan,AF,AFG,38041757.0,Asia,2.670749
3,2020-08-24,24,8,2020,0,0,Afghanistan,AF,AFG,38041757.0,Asia,2.484112
4,2020-08-23,23,8,2020,105,2,Afghanistan,AF,AFG,38041757.0,Asia,2.484112
...,...,...,...,...,...,...,...,...,...,...,...,...
38906,2020-03-25,25,3,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN
38907,2020-03-24,24,3,2020,0,1,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN
38908,2020-03-23,23,3,2020,0,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN
38909,2020-03-22,22,3,2020,1,0,Zimbabwe,ZW,ZWE,14645473.0,Africa,NaN


In [7]:
covid.rename(columns = {'countriesAndTerritories': 'Country'}, inplace = True)
covid.rename(columns = {'dateRep': 'DateRep', 'day':'Day', 'month':'Month','year':'Year','cases':'Cases','deaths':'Deaths', 'popData2019':'Population', 'continentExp':'Continent'}, inplace = True)
covid.replace('CANADA','Canada', inplace=True)
covid.replace(['United_States_of_America', 'United_Kingdom', 'South_Korea', 'Saudi_Arabia', 'South_Africa', 'United_Arab_Emirates', 'Dominican_Republic'],['USA', 'UK', 'S.Korea', 'S.Arabia', 'S.Africa', 'UAE', 'Dom.Rep.'], inplace = True)
covid.drop(columns = ['geoId', 'countryterritoryCode', 'Cumulative_number_for_14_days_of_COVID-19_cases_per_100000'], inplace = True)
# covid
covid[covid['Country'] == 'Canada']
# covid[covid['Country'] == 'Canada'].sort_values('DateRep').head(50)

,DateRep,Day,Month,Year,Cases,Deaths,Country,Population,Continent
0,2020-08-27,27,8,2020,55,4,Afghanistan,38041757.0,Asia
1,2020-08-26,26,8,2020,1,0,Afghanistan,38041757.0,Asia
2,2020-08-25,25,8,2020,71,10,Afghanistan,38041757.0,Asia
3,2020-08-24,24,8,2020,0,0,Afghanistan,38041757.0,Asia
4,2020-08-23,23,8,2020,105,2,Afghanistan,38041757.0,Asia
...,...,...,...,...,...,...,...,...,...
38906,2020-03-25,25,3,2020,0,0,Zimbabwe,14645473.0,Africa
38907,2020-03-24,24,3,2020,0,1,Zimbabwe,14645473.0,Africa
38908,2020-03-23,23,3,2020,0,0,Zimbabwe,14645473.0,Africa
38909,2020-03-22,22,3,2020,1,0,Zimbabwe,14645473.0,Africa


In [27]:
# GENERATE THE CUMULATIVE COUNTS
covid_cumulative = covid.groupby(['DateRep','Country']).sum().groupby('Country').cumsum()

covid_cumulative['Date'] = covid_cumulative.index.get_level_values(0)
covid_cumulative['Countries'] = covid_cumulative.index.get_level_values(1)
covid_cumulative.drop(columns=['Population', 'Day', 'Month', 'Year'], inplace = True)
covid_cumulative[covid_cumulative['Countries'] == 'Canada']

,,Cases,Deaths,Date,Countries
DateRep,Country,,,,
2019-12-31,Canada,0,0,2019-12-31,Canada
2020-01-01,Canada,0,0,2020-01-01,Canada
2020-01-02,Canada,0,0,2020-01-02,Canada
2020-01-03,Canada,0,0,2020-01-03,Canada
2020-01-04,Canada,0,0,2020-01-04,Canada
...,...,...,...,...,...
2020-08-23,Canada,124629,9071,2020-08-23,Canada
2020-08-24,Canada,124896,9073,2020-08-24,Canada
2020-08-25,Canada,125647,9083,2020-08-25,Canada


In [18]:
# RE-JOIN TO GET COUNTRY'S POPULATIONS
covid_cumulative = covid_cumulative.merge(covid[['DateRep', 'Country', 'Population', 'Continent']], how='inner', left_index=True, right_on=['DateRep', 'Country']).set_index(['DateRep', 'Country'])
covid_cumulative

Cases  Deaths       Date       Countries  \
DateRep    Country                                                    
2019-12-31 Afghanistan         0       0 2019-12-31     Afghanistan   
           Algeria             0       0 2019-12-31         Algeria   
           Armenia             0       0 2019-12-31         Armenia   
           Australia           0       0 2019-12-31       Australia   
           Austria             0       0 2019-12-31         Austria   
...                          ...     ...        ...             ...   
2020-08-27 Vietnam          1034      30 2020-08-27         Vietnam   
           Western_Sahara    766       1 2020-08-27  Western_Sahara   
           Yemen            1930     560 2020-08-27           Yemen   
           Zambia          11376     282 2020-08-27          Zambia   
           Zimbabwe         6251     179 2020-08-27        Zimbabwe   

                           Population Continent  
DateRep    Country                               
2019-12-31 Afghanistan     38041757.0      Asia  
           Algeria         43053054.0    Africa  
           Armenia          2957728.0    Europe  
           Australia       25203200.0   Oceania  
           Austria          8858775.0    Europe  
...                               ...       ...  
2020-08-27 Vietnam         96462108.0      Asia  
           Western_Sahara    582458.0    Africa  
           Yemen           29161922.0      Asia  
           Zambia          17861034.0    Africa  
           Zimbabwe        14645473.0    Africa  

[38911 rows x 6 columns]

In [19]:
covid_cumulative = covid_cumulative.reset_index().drop(columns=['DateRep', 'Country'])
covid_cumulative.to_excel('covid_a_master_cumulative_table.xlsx')

In [20]:
covid_cumulative

,Cases,Deaths,Date,Countries,Population,Continent
0,0,0,2019-12-31,Afghanistan,38041757.0,Asia
1,0,0,2019-12-31,Algeria,43053054.0,Africa
2,0,0,2019-12-31,Armenia,2957728.0,Europe
3,0,0,2019-12-31,Australia,25203200.0,Oceania
4,0,0,2019-12-31,Austria,8858775.0,Europe
...,...,...,...,...,...,...
38906,1034,30,2020-08-27,Vietnam,96462108.0,Asia
38907,766,1,2020-08-27,Western_Sahara,582458.0,Africa
38908,1930,560,2020-08-27,Yemen,29161922.0,Asia
38909,11376,282,2020-08-27,Zambia,17861034.0,Africa
